##Assignment 11 EVA

This contains code that creates a modified ResNet DNN in PyTorch with best LR search using LR Range Test and training using Once Cycle Policy

In [0]:
from google.colab import drive
drive.mount("/content/gdrive/", force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [0]:
import sys
sys.path.append('/content/gdrive/My Drive/Colab Notebooks/EVA4/S11')

Install Albumentations

In [0]:
!pip install albumentations==0.4.5

     |████████████████████████████████| 122kB 4.7MB/s 
     |████████████████████████████████| 634kB 14.8MB/s 
  Created wheel for albumentations: filename=albumentations-0.4.5-cp36-none-any.whl size=64378 sha256=fa0a4a5afd7290d0d558f6f1f23e99eeb1064f714881254d9c38ee87c574f3cb
  Stored in directory: /root/.cache/pip/wheels/f0/a0/61/e50f93165a5ec7e7f5d65064e513239505bc4c06d2289557d3
  Created wheel for imgaug: filename=imgaug-0.2.6-cp36-none-any.whl size=654020 sha256=ed51968e8dd7f6bc332aab3129a4735951cebe82041bc0d4ab0b98bd1c9c45ee
  Stored in directory: /root/.cache/pip/wheels/97/ec/48/0d25896c417b715af6236dbcef8f0bed136a1a5e52972fc6d0
Successfully built albumentations imgaug
  Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9
  Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12


Import libraries

In [0]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
#import torchvision.transforms as transforms
from torchsummary import summary
from tqdm import tqdm
import torch.backends.cudnn as cudnn

from transform.albu_transforms import *
import datasetloader as d
from models.A11DNN import *
from train_test import train, test

Load the data

In [0]:
# setting up random seed and processing device
SEED = 1
cuda = torch.cuda.is_available()
if cuda:
    device = "cuda"
    torch.cuda.manual_seed(SEED)
else:
    device = "cpu"
    torch.manual_seed(SEED)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

## getting training data loader
trainloader = d.get_dataloader_CIFAR10(True, cuda, transform=transform_train_albu)

## getting test data loader
testloader = d.get_dataloader_CIFAR10(False, cuda, transform=transform_test_albu)

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


Initialize the network architecture

In [0]:
!pip install torchsummary
from torchsummary import summary
net = A11DNN()
net = net.to(device)
if cuda:
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True
summary(net, input_size=(3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,792
       BatchNorm2d-2           [-1, 64, 32, 32]             128
              ReLU-3           [-1, 64, 32, 32]               0
            Conv2d-4          [-1, 128, 32, 32]          73,856
         MaxPool2d-5          [-1, 128, 16, 16]               0
       BatchNorm2d-6          [-1, 128, 16, 16]             256
              ReLU-7          [-1, 128, 16, 16]               0
            Conv2d-8          [-1, 128, 16, 16]         147,456
       BatchNorm2d-9          [-1, 128, 16, 16]             256
           Conv2d-10          [-1, 128, 16, 16]         147,456
      BatchNorm2d-11          [-1, 128, 16, 16]             256
      ResNetBlock-12          [-1, 128, 16, 16]               0
           Conv2d-13          [-1, 256, 16, 16]         295,168
        MaxPool2d-14            [-1, 25

/content/gdrive/My Drive/Colab Notebooks/EVA4/S11/models/A11DNN.py:86: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x9)


LR Range Test

In [0]:
lr_min = 0.0001
lr_max = 0.2
lr_step = 0.001

# loss function
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0, momentum=0.9, weight_decay=5e-4)
accuracies = []
epoch = 1
# Model Training and Evaluation
for lr_t in np.arange(lr_min, lr_max, lr_step):
    avg_train_loss, accuracy = train(net, device, trainloader, optimizer, epoch, criterion)
    accuracies.append(accuracy)
    optimizer = optim.SGD(net.parameters(), lr=lr_t, momentum=0.9, weight_decay=5e-4)
    epoch = epoch + 1
    #test(net, device, testloader, criterion)

  0%|          | 0/391 [00:00<?, ?it/s]/content/gdrive/My Drive/Colab Notebooks/EVA4/S11/models/A11DNN.py:86: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x9)
epoch=1 Loss=3.2568607330322266 Batch_id=390 Accuracy=12.04: 100%|██████████| 391/391 [00:40<00:00,  9.61it/s]
epoch=2 Loss=1.7874586582183838 Batch_id=390 Accuracy=27.76: 100%|██████████| 391/391 [00:39<00:00,  9.81it/s]
epoch=3 Loss=1.543853998184204 Batch_id=390 Accuracy=38.61: 100%|██████████| 391/391 [00:40<00:00,  9.67it/s]
epoch=4 Loss=1.4306228160858154 Batch_id=390 Accuracy=45.86: 100%|██████████| 391/391 [00:40<00:00,  9.71it/s]
epoch=5 Loss=1.5206619501113892 Batch_id=390 Accuracy=50.34: 100%|██████████| 391/391 [00:40<00:00,  9.56it/s]
epoch=6 Loss=1.2032674551010132 Batch_id=390 Accuracy=54.26: 100%|██████████| 391/391 [00:40<00:00,  9.57it/s]
epoch=7 Loss=1.4755187034606934 Batch_id=390 Accuracy=57.01: 100%|███████

One Cycle Policy